In [1]:
import ipywidgets as widgets
import numpy as np
from IPython.display import HTML


HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
system = widgets.Select(
    options=['Nucleogenic/Cosmogenic/Air', 'Nucleogenic/Mantle/Air', 'Cosmogenic/Mantle/Air'],
    value='Nucleogenic/Cosmogenic/Air',
    rows=4,
    description='Ternary System:',
    style = {'description_width': 'initial'},
    disabled=False
)

mineral = widgets.Select(
    options=['quartz', 'pyroxene'],
    value='quartz',
    rows=3,
    description='Mineral:',
    style = {'description_width': 'initial'},
    disabled=False
)

SysInputs = widgets.GridBox(children=[system, mineral],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto auto auto',
            grid_template_columns='50% 50%',
            grid_template_areas='''
            "system mineral"
            '''
        )
)

Ne20 = widgets.FloatText(
    value=9048,
    description='Neon-20',
    disabled=False
)

Ne21 = widgets.FloatText(
    value=27,
    description='Neon-21',
    disabled=False
)

Ne22 = widgets.FloatText(
    value=925,
    description='Neon-22',
    disabled=False
)

NeInputs = widgets.GridBox(children=[Ne20, Ne21, Ne22],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto auto auto',
            grid_template_columns='33% 33% 33%',
            grid_template_areas='''
            "Ne20 Ne21 Ne22"
            '''
        )
)

In [3]:
def deconvolve(system):
    
    out = widgets.Output(layout={'border': '1px solid black'})
    
    with out:
        print(system)
        print("Input: (",round(Ne20.value,5)," Ne20) (", round(Ne21.value,5)," Ne21) (", round(Ne22.value,5)," Ne22)")
    
    comp = []
    comp_name = []
    comp_option = []
    neon_20_22_comp = []
    neon_21_22_comp = []
    neon_22_frac = []
    neon_20_frac = []
    neon_21_frac = []
    neon_22_norm = []
    neon_20_norm = []
    neon_21_norm = []
    neon_22_sig = []
    neon_20_sig = []
    neon_21_sig = []

    if system == 'Nucleogenic/Cosmogenic/Air':
        comp.append(0)
        comp.append(1)
        comp.append(2)
        comp_name.append('Nucleogenic')
        comp_name.append('Cosmogenic')
        comp_name.append('Air')

    elif system == 'Nucleogenic/Mantle/Air':
        comp.append(0)
        comp.append(3)
        comp.append(2)
        comp_name.append('Nucleogenic')
        comp_name.append('Mantle')
        comp_name.append('Air')

    else:
        comp.append(3)
        comp.append(1)
        comp.append(2)
        comp_name.append('Cosmogenic')
        comp_name.append('Mantle')
        comp_name.append('Air')

    #nucleogenic
    comp_option.append([.99999,.000005,.000005])

    #cosmogenic (Niedermann 2002)

    if mineral == 'quartz':
        #quartz
        comp_option.append([0.4582741396,0.3666193117,0.1751065487])
    else:
        #pyroxene
        comp_option.append([0.301659125188537,0.324283559577677,0.374057315233786])

    #air
    comp_option.append([.9048,.0027,.0925])
    
    for i in range(3):
        comp[i] = comp_option[comp[i]]
        neon_20_22_comp.append(comp[i][0]/comp[i][2])
        neon_21_22_comp.append(comp[i][1]/comp[i][2])
        
    neon_20_22 = Ne20.value/Ne22.value
    neon_21_22 = Ne21.value/Ne22.value

    data_matrix = np.array([[neon_20_22], [neon_21_22], [1]])

    model_matrix = np.array([[neon_20_22_comp[0], neon_20_22_comp[1], neon_20_22_comp[2]], [neon_21_22_comp[0], neon_21_22_comp[1],
                                                                                      neon_21_22_comp[2]], [1, 1, 1]])    
    inv_model = np.linalg.inv(model_matrix)
    
    results_matrix = np.matmul(inv_model, data_matrix)

    for i in range(3):
        neon_22_frac.append(results_matrix[i])
        neon_22_norm.append(results_matrix[i]/sum(results_matrix))
        neon_20_frac.append(neon_22_frac[i]*neon_20_22_comp[i])
        neon_21_frac.append(neon_22_frac[i]*neon_21_22_comp[i])
                            
    for i in range(3):
        neon_20_norm.append(neon_20_frac[i]/sum(neon_20_frac))
        neon_21_norm.append(neon_21_frac[i]/sum(neon_21_frac))

        neon_22_sig.append(Ne22.value*neon_22_norm[i])
        neon_20_sig.append(Ne20.value*neon_20_norm[i])
        neon_21_sig.append(Ne21.value*neon_21_norm[i])
        with out:
            print(comp_name[i],": (",round(neon_20_sig[i][0],5)," Ne20) (", round(neon_21_sig[i][0],5)," Ne21) (", round(neon_22_sig[i][0],5)," Ne22)")
        #return neon_20_sig[i], neon_21_sig[i], neon_22_sig[i]

    with out:
        print(" ")
        
    return out

In [4]:
clicker = widgets.Button(
    description='Calculate',
    disabled=False,
    tooltip='Calculate deconvolution',
    icon='check'
)

def on_button_clicked(self):
    out = deconvolve(system.value)
    display(out)
    
clicker.on_click(on_button_clicked)

In [5]:
display(SysInputs)
display(NeInputs)
display(clicker)

GridBox(children=(Select(description='Ternary System:', options=('Nucleogenic/Cosmogenic/Air', 'Nucleogenic/Ma…

GridBox(children=(FloatText(value=9048.0, description='Neon-20'), FloatText(value=27.0, description='Neon-21')…

Button(description='Calculate', icon='check', style=ButtonStyle(), tooltip='Calculate deconvolution')